# Assignment#5 - Lecture5: Lab2
## Lab 2: Full-Adder Quantum Circuit 

In [1]:
import cirq
from cirq.ops import *

In [2]:
# q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]

In [3]:
def OR(a, b, c):
    yield [CNOT(a, b)]
    yield [CNOT(a, c)]
    yield [CCNOT(a, b, c)]

In [4]:
def XOR(a, b, s):
    yield [CNOT(b, s)]
    yield [CNOT(a, s)]

In [5]:
def AND(a, b, c):
    yield [CCNOT(a, b, c)]

In [6]:
def HA(a, b, s, c):
    yield XOR(a, b, s)
    yield AND(a, b, c)
    
q_a, q_b, q_s, q_c = [cirq.GridQubit(i, 0) for i in range(4)]
ha = cirq.Circuit()
ha.append(HA(q_a, q_b, q_s, q_c))
print(ha)

(0, 0): ───────@───@───
               │   │
(1, 0): ───@───┼───@───
           │   │   │
(2, 0): ───X───X───┼───
                   │
(3, 0): ───────────X───


### Logic circuit

In [8]:
def FA(a, b, x, s, c, s1, c1, c2):
    yield HA(a, b, s1, c1)
    yield HA(s1, x, s, c2)
    yield AND(c1, c2, c)

a = cirq.GridQubit(0, 0)
b = cirq.GridQubit(1, 0)
x = cirq.GridQubit(2, 0)

s, c, s1, c1, c2 = [cirq.GridQubit(i, 0) for i in range(3, 8)]
ms = [measure(s, key='sum')]
mc = [measure(c, key='carry out')]
fullAdder = cirq.Circuit(FA(a, b, x, s, c, s1, c1, c2), ms, mc)
print(fullAdder)

           ┌──┐       ┌──┐   ┌─────────┐
(0, 0): ──────────@────@────────────────────────────────────────
                  │    │
(1, 0): ────@─────┼────@────────────────────────────────────────
            │     │    │
(2, 0): ────┼@────┼────┼──────@─────────────────────────────────
            ││    │    │      │
(3, 0): ────┼X────┼────┼X─────┼M('sum')─────────────────────────
            │     │    ││     │
(4, 0): ────┼─────┼────┼┼─────┼────────────X───M('carry out')───
            │     │    ││     │            │
(5, 0): ────X─────X────┼@─────@────────────┼────────────────────
                       │      │            │
(6, 0): ───────────────X──────┼────────────@────────────────────
                              │            │
(7, 0): ──────────────────────X────────────@────────────────────
           └──┘       └──┘   └─────────┘


### K-map

In [9]:
def FA(a, b, s, c_in, c_out, d, e, f, g, h):
    # s
    yield [XOR(a, b, d)]
    yield [XOR(d, c_in, s)]
    # c_out
    yield [AND(a, c_in, e)]
    yield [AND(b, c_in, f)]
    yield [AND(a, b, g)]
    
    yield [OR(e, f, h)]
    yield [OR(g, h, c_out)]

In [10]:
a, b, s, c_in, c_out, d, e, f, g, h = [cirq.GridQubit(i, 0) for i in range(10)]
a = cirq.GridQubit(0, 0)
ms = [measure(s, key='sum')]
mc = [measure(c_out, key='carry out')]
fa = cirq.Circuit(FA(a, b, s, c_in, c_out, d, e, f, g, h), ms, mc)
print(fa)

           ┌──┐       ┌──┐   ┌─────────┐   ┌──┐
(0, 0): ──────────@─────@───────────────────@──────────────────────────────────────────
                  │     │                   │
(1, 0): ────@─────┼─────┼─────@─────────────@──────────────────────────────────────────
            │     │     │     │             │
(2, 0): ────┼X────┼────X┼─────┼M('sum')─────┼──────────────────────────────────────────
            ││    │    ││     │             │
(3, 0): ────┼@────┼────┼@─────@─────────────┼──────────────────────────────────────────
            │     │    ││     │             │
(4, 0): ────┼─────┼────┼┼─────┼─────────────┼─────────────────X───X───M('carry out')───
            │     │    ││     │             │                 │   │
(5, 0): ────X─────X────@┼─────┼─────────────┼─────────────────┼───┼────────────────────
                        │     │             │                 │   │
(6, 0): ────────────────X─────┼─────────────┼@────@───@───────┼───┼────────────────────
                